# Player Season Batting Summary
Calculates season-level batting performance, salary efficiency, and derived metrics per player.

In [ ]:
dbutils.widgets.text('season_year', '', 'Season Year')
season_year = dbutils.widgets.get('season_year')

In [ ]:
game_stats = spark.read.table('raw.trouze.raw_game_stats')
players = spark.read.table('raw.trouze.raw_players')
salaries = spark.read.table('raw.trouze.raw_salaries')
teams = spark.read.table('raw.trouze.raw_teams')

In [ ]:
from pyspark.sql import functions as F

player_season_batting = (
    game_stats
    .filter(F.year(F.col('game_date')) == int(season_year))
    .groupBy('player_id')
    .agg(
        F.count('game_id').alias('games_played'),
        F.sum('at_bats').alias('total_at_bats'),
        F.sum('hits').alias('total_hits'),
        F.sum('home_runs').alias('total_home_runs'),
        F.sum('rbi').alias('total_rbi'),
        F.sum('stolen_bases').alias('total_stolen_bases'),
        F.sum('walks').alias('total_walks'),
        F.sum('strikeouts').alias('total_strikeouts')
    )
)

In [ ]:
player_salaries = (
    salaries
    .filter(F.col('year') == int(season_year))
    .select('player_id', 'team_id', 'salary')
)

In [ ]:
result = (
    player_season_batting
    .join(players.select('player_id', 'first_name', 'last_name', 'position'), on='player_id', how='inner')
    .join(player_salaries, on='player_id', how='left')
    .join(teams.select('team_id', 'team_name', 'league', 'division'), on='team_id', how='left')
    .withColumn('salary', F.coalesce(F.col('salary'), F.lit(0)))
    .withColumn('batting_avg', F.when(F.col('total_at_bats') > 0, F.col('total_hits') / F.col('total_at_bats')).otherwise(0))
    .withColumn('on_base_pct', F.when((F.col('total_at_bats') + F.col('total_walks')) > 0, (F.col('total_hits') + F.col('total_walks')) / (F.col('total_at_bats') + F.col('total_walks'))).otherwise(0))
    .withColumn('cost_per_hit', F.when(F.col('total_hits') > 0, F.col('salary') / F.col('total_hits')).otherwise(None))
    .select(
        'player_id', 'first_name', 'last_name', 'position',
        'team_id', 'team_name', 'league', 'division',
        'games_played', 'total_at_bats', 'total_hits', 'total_home_runs',
        'total_rbi', 'total_stolen_bases', 'total_walks', 'total_strikeouts',
        'batting_avg', 'on_base_pct', 'salary', 'cost_per_hit'
    )
)

In [ ]:
spark.sql(f"""
  MERGE INTO analytics.gold.player_season_batting_summary AS target
  USING {{result_view}} AS source
  ON target.player_id = source.player_id AND target.season_year = {season_year}
  WHEN MATCHED THEN UPDATE SET *
  WHEN NOT MATCHED THEN INSERT *
""")
print('Done')